# Metal Music Sampling

References: 

https://github.com/shubham3121/music-generation-using-rnn 

https://www.hackerearth.com/blog/developers/jazz-music-using-deep-learning/

https://pyguitarpro.readthedocs.io/en/stable/index.html

## Imports

In [1]:
import guitarpro
from guitarpro import *
from matplotlib import pyplot as plt
import numpy as np
import os
import pickle

from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import Activation, Dense, LSTM, Dropout, Flatten

from _Decompressor import decompress_track

## Constants

In [2]:
SEQUENCE_LENGTH = 100


# PITCH[i] = the pitch associated with midi note number i.
# For example, PITCH[69] = 'A4'
PITCH = {val : str(GuitarString(number=0, value=val)) for val in range(128)}
# MIDI[string] = the midi number associated with the note described by string.
# For example, MIDI['A4'] = 69.
MIDI  = {str(GuitarString(number=0, value=val)) : val for val in range(128)}

## Load data

In [3]:
with open('notes', 'rb') as filepath:
    notes = pickle.load(filepath)

with open('note_int_conversions', 'rb') as filepath:
    note_to_int = pickle.load(filepath)
    int_to_note = pickle.load(filepath)
    
n_vocab = len(note_to_int)

## Model generation function (from _Training.ipynb)

In [4]:
def create_network(network_in, n_vocab): 
    """Create the model architecture"""
    model = Sequential()
    model.add(LSTM(128, input_shape=network_in.shape[1:], return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(128, return_sequences=True))
    model.add(Flatten())
    model.add(Dense(256))
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam')

    return model

## Note generation

In [5]:
def get_inputSequences(notes, note_to_int, n_vocab):
    """ Prepare the sequences used by the Neural Network """

    generation_input = []
    for i in range(0, len(notes) - SEQUENCE_LENGTH, 1):
        sequence_in = notes[i:i + SEQUENCE_LENGTH]
        generation_input.append([note_to_int[char] for char in sequence_in])
    
    generation_input = np.reshape(generation_input, (len(generation_input), SEQUENCE_LENGTH, 1))
    
    return (generation_input)

In [6]:
def generate_notes(model, network_input, note_to_int, n_vocab):
    """ Generate notes from the neural network based on a sequence of notes """
    # Pick a random integer
    start = np.random.randint(0, len(network_input)-1)

    # Invert the note_to_int dictionary to get the int_to_note dictionary.
    int_to_note = inv_map = {v: k for k, v in note_to_int.items()}
    
    # pick a random sequence from the input as a starting point for the prediction
    '''
    TODO: Look into the pattern generation stuff.
    '''
    pattern = list(network_input[start])
    #pattern = list(np.random.choice(n_vocab, (100,1))) # Generate an iniitial sequence at random from n_vocab.
    prediction_output = []
    
    print('Generating notes........')

    # generate 500 notes
    for note_index in range(500):
        prediction_input = np.reshape(pattern, (1, len(pattern), 1))
        prediction_input = prediction_input / float(n_vocab)

        prediction = model.predict(prediction_input, verbose=0)

        # Predicted output is the argmax(P(h|D))
        index = np.argmax(prediction)
        # Mapping the predicted interger back to the corresponding note
        result = int_to_note[index]
        # Storing the predicted output
        prediction_output.append(result)

        pattern.append(np.array([index])) # Fixed this code to make sure the new element matched the datatype of the existing elements.
        # Next input to the model
        pattern = pattern[1:len(pattern)]

    print('Notes Generated.')
    return prediction_output

In [7]:
""" Generate a .gp5 tab file """

'''
#load the notes used to train the model
with open('data/notes', 'rb') as filepath:
    notes = pickle.load(filepath)

# Get all pitch names
pitchnames = sorted(set(item for item in notes))
# Get all pitch names
n_vocab = len(set(notes))
'''

print('Initiating music generation process.......')

#network_input = get_inputSequences(notes, pitchnames, n_vocab)
generation_input = get_inputSequences(notes, note_to_int, n_vocab)

normalized_input = generation_input / float(n_vocab)
model = create_network(normalized_input, n_vocab)
print('Loading Model weights.....')
model.load_weights('weights.best.music3.hdf5')
print('Model Loaded')
prediction_output = generate_notes(model, generation_input, note_to_int, n_vocab)

Initiating music generation process.......


KeyError: ('7_dim5', 8, False)

In [ ]:
def thirty_seconds_to_duration(count):
    if count % 3 == 0:
        # If the note is dotted, do 32 / (i * 2/3), and return isDotted = True.
        return (48//count, True)
    else:
        # If the note is not dotted, to 32 / i, and return isDotted = False.
        return (32//count, False)

def quantize_thirty_seconds(value):

    # 32nd-note values of each fundamental type of note (not including 64th-notes, of course).
    vals = np.array([32, # whole
                     24, # dotted half
                     16, # half
                     12, # dotted quarter
                     8,  # quarter
                     6,  # dotted eigth
                     4,  # eigth
                     3,  # dotted sixteenth
                     2,  # sixteenth
                     1]) # thirty-second
    
    list_out = []

    for v in vals:
        if v <= value:
            list_out.append(thirty_seconds_to_duration(v))
            value -= v
            
    return np.array(list_out)

## Adjust prediction output to 4/4 time

In [ ]:
# This will be the prediction output
new_prediction_output = []


time = 0
for beat in prediction_output:
    
    # Calculate the fraction of a measure encompassed by the current beat / chord.
    beat_time = (1 / beat[1]) * (1 + 0.5 * beat[2])
    
    # Calculate the fraction of a measure taken up by all notes in the measure.
    # Calculate any residual time to see if this measure (in 4/4 time) is longer than 1 measure.
    measure_time = time + beat_time
    leftover_time = (measure_time) % 1
    
    # If the measure count (i.e., the measure integer) has changed and there is significant left-over beat time:
    if (int(measure_time) > int(time)) and (leftover_time > 1/128):
        
        # Calculate the initial 32nd notes encompassed by this beat in the current measure.
        this_measure_thirty_seconds = int(32 * (1 - time % 1))
        # Calculate the remaining 32nd notes encompassed by this beat in the next measure.
        next_measure_thirty_seconds = int(32 * leftover_time)
        
        # Get the Duration object parameters for this measure and the next measure.
        this_measure_durations = quantize_thirty_seconds(this_measure_thirty_seconds)
        next_measure_durations = quantize_thirty_seconds(next_measure_thirty_seconds)
        
        
        #print(f'{{ {32 / beat[1]}')
        for duration_idx, duration in enumerate(this_measure_durations):
            time += (1 / duration[0]) * (1 + 0.5 * duration[1])
            
            #print(time, '\t', time * 32)
                
            chord = beat[0] if duration_idx == 0 else 'tied'
            
            new_prediction_output.append((chord, duration[0], duration[1]))
            
            
        for duration in next_measure_durations:
            time += (1 / duration[0]) * (1 + 0.5 * duration[1])
            
            #print(time, '\t', time * 32)
            
            new_prediction_output.append(('tied', duration[0], duration[1]))
            
               
        continue
    
    
    time += beat_time
    new_prediction_output.append((beat[0], beat[1], beat[2]))
    
    #print(time, '\t', time * 32)


'''
time = 0
time2 = 0
idx = 0

for idx2, beat2 in enumerate(new_prediction_output[:100]):
    beat = prediction_output[idx]
    
    if time == time2:
        print(beat[0], '\t', time, '\t\t', beat2[0], '\t', time2)
        
        idx += 1
        
        time += (1 / beat[1]) * (1 + 0.5 * beat[2])
    
    else:
        print('\t\t\t\t', beat2[0], '\t', time2)

    
    
    time2 += (1 / beat2[1]) * (1 + 0.5 * beat2[2])
''';

## Separate prediction output notes into measures

In [ ]:
# Use the previously calculated cumulative time as the number of measures in the new 4/4 song.
num_measures = int(np.ceil(time))

song = np.empty(num_measures, dtype=object)

time = 0
m_idx = 0

timestamps = []

for beat in new_prediction_output:
    #print(time)
    timestamps.append(time)
    
    m_idx = int(time)
    
    if song[m_idx] is None:
        
        song[m_idx] = [beat]
    else:
        song[m_idx].append(beat)
    
    
    time += (1 / beat[1]) * (1 + 0.5 * beat[2])
    
    
print(f'4/4 adjusted correctly: {set(range(num_measures)).issubset(set(timestamps))}')

## Figure out the necessary guitar tuning for the produced song

In [ ]:
# Get the tuning (i.e., the lowest note) of the song:

'''
pitchnames = set([x[0] for x in prediction_output])
pitchnames.discard('rest')
pitchnames.discard('tied')
pitchnames.discard('dead')


# Standard tuning
tuning = {1: MIDI['E4'],
          2: MIDI['B3'],
          3: MIDI['G3'],
          4: MIDI['D3'],
          5: MIDI['A2'],
          6: MIDI['E2']}

# Get the lowest note in the output.
# The highest tuning allowed will be standard tuning.
lowest_note = min([MIDI[x.split('_')[0]] for x in pitchnames])
lowest_note = min(lowest_note, MIDI['E2'])

if lowest_note <= MIDI['B1']:
    # 7-string guitar case
    tuning[7] = MIDI['B1']
    drop = MIDI['B1'] - lowest_note
else:
    # drop the tuning by however much is necessary.
    drop = MIDI['E2'] - lowest_note
    
tuning = {k: v - drop for k, v in tuning.items()}
tuning
'''

# Standard tuning
tuning = {1: MIDI['E4'],
          2: MIDI['B3'],
          3: MIDI['G3'],
          4: MIDI['D3'],
          5: MIDI['A2'],
          6: MIDI['E2']}
tuning

## Lastly, save the song to a .gp5 file

In [ ]:
guitarpro.write(decompress_track(song, tuning), '_generation.gp5')
print('Finished')

'''
TODO: Find out which song / where test sequence sources are coming from.

TODO: Normalize sequences based on lowest tuning notes to reduce overfitting.
TODO: After implementing tuning normalization, decide the tuning of the generated song based on the tuning of the input sequence.

TODO: Consider filtering out bass parts.

TODO: Train with a wider variety of songs once the generation process has been examined.
TODO: If training does not yield true creativity, try generating sequences using the random sequence initialization.
''';